<img src="http://www.exalumnos.usm.cl/wp-content/uploads/2015/06/Isotipo-Negro.gif" title="Title text" width="20%" height="20%" />


<hr style="height:2px;border:none"/>
<h1 align='center'> INF-395 Redes Neuronales Artificiales I-2018 </h1>

<H3 align='center'> Tarea 1 - Redes Neuronales y *Deep Learning* </H3>
<hr style="height:2px;border:none"/>

**Temas**  

* Entrenamiento de redes *Feed-Forward* vı́a GD y variantes (SGD, mini-*batches*), *momentum*, regularización y tasa de aprendizaje adaptiva.
* Rol de capas ocultas y mayor profundidad (*Deep Learning*).
* Diseño y entrenamiento de Redes Neuronales Convolucionales (CNNs).
* Aplicaciones de las Redes Neuronales Convolucionales
* Técnicas de regularización: *Dropout* y *Batch Normalization* 

<hr style="height:2px;border:none"/>
La tarea se divide en secciones:

[1.](#primero) Red Neuronal *Feed Forward* para Detectar Exoplanetas  
[2.](#segundo) *Deep Networks*  
[3.](#tercero) Redes Convolucionales en Imágenes  
[4.](#cuarto) CNN *vs* RNN Prediciendo el Ozono Atmosférico

## 1. Red Neuronal *Feed Forward* para Detectar Exoplanetas
---

Las Redes Neuronales Artificiales (ANN) son un tipo de modelo de aprendizaje que, con el suficiente número de capas, debiera poder aproximar cualquier función. Hoy en día se han aplicado a numerosos problemas, ya que tienen la ventaja de ser bastante dinámicas en su construcción y poder ser entrenadas con unicamente *backpropagation*, obteniendo un buen desempeño. Los modelos de aprendizaje son aplicados a diferentes ámbitos en donde, en algunos casos, automatizan extensivos procesos manuales de expertos, como lo que veremos a continuación.

<img src="https://www.cfa.harvard.edu/~avanderb/tutorial/HAT-P-3b.gif" title="YOLO" width="30%"/>

En esta sección trabajaremos con un dataset de astronomía, **Kepler KOI** [[1]](#refs), en el cual podrán experimentar con la detección de exoplanetas a través de *features* extraídas de las curvas de luz (tránsito de los planetas sobre su estrella madre) y metadatos de la observación (planeta, estrella y curvas de luz). Para más información sobre los metadatos visite el siguiente __[link](https://exoplanetarchive.ipac.caltech.edu/docs/API_kepcandidate_columns.html)__. La tarea es determinar si un posible candidato a exoplaneta (en base a una previa identificación/selección de eventos que sobrepasan un cierto *treshold*) corresponde efectivamente a un exoplaneta o no.

---
En primer lugar cargue todos los datos trabajados por [[2]](#refs), los cuales se encuentran adjuntados con el enunciado. Además separe en entrenamiento y pruebas.

In [ ]:
import pandas  as pd
import numpy as np
df_sets = pd.read_csv("./koi_sets_unb.csv")
mask_train = (df_sets["Set"] == "Train").values
mask_test = (df_sets["Set"] == "Test").values
df_labels = pd.read_csv("./koi_labels.csv")
df_X = pd.read_csv("./koi_light_curves_X.csv")
df_labels_train = df_labels[mask_train]
df_labels_test = df_labels[mask_test]
df_X_train = df_X[mask_train]
df_X_test = df_X[mask_test]

A continuación cree las matrices que trabajará y servirán para entrenar su modelo neuronal, la etiqueta binaria (0 o 1) y la matriz numérica de entrada, imputando el estadístico de la mediana sobre los valores nulos.

In [ ]:
y_train = ((df_labels_train["NExScI Disposition"]=="CONFIRMED")*1).values
y_test = ((df_labels_test["NExScI Disposition"]=="CONFIRMED")*1).values
df_X_train = df_X_train.reset_index(drop=True)
df_X_test = df_X_test.reset_index(drop=True)
df_X_train.fillna(df_X_train.median(), inplace=True)
df_X_test.fillna(df_X_test.median(), inplace=True)
X_train = df_X_train.values[:,1:]
X_test = df_X_test.values[:,1:]

**a)** Explore los datos trabajados, ya sea con estadísticos simples o con gráficos como histogramas y/o boxplots. Comente sobre el problema enfrentado, es decir, la tarea de transformar un vector $X$ en un valor categórico (0 o 1).

In [ ]:
columns_names = df_X_train.columns[1:]
print(columns_names)
print(X_train.shape)
print(X_test.shape)
df_X_train.describe()

**b)** Escale los datos para ser trabajados por el modelo de aprendizaje, indique la importancia de éste paso. Además cree un conjunto de validación extrayendo un cierto porcentaje del conjunto de entrenamiento, por ejemplo el 20% manteniendo el desbalanceo de clases (*split* stratificado).

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_val_scaled = ...
X_test_scaled =  scaler.transform(X_test)

**c)** Muestre en un gráfico la función objetivo (*cross entropy*) para el conjunto de entrenamiento y de validación *vs* número de *epochs* de entrenamiento, para una red *feedforward* de 3 capas, con 256 unidades ocultas y función de activación sigmoidal. Entrene la red usando gradiente descendente estocástico con tasa de aprendizaje (*learning rate*) 0.01 y 100 *epochs* de entrenamiento. Comente. Si observa divergencia durante el entrenamiento, determine si esto ocurre para cada repetición del experimento. Compare el efecto de variar la función de activación a **ReLU** ¿Qué observa en la convergencia del modelo?

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD
model = Sequential()
model.add(Dense(256, input_dim=X_train_scaled.shape[1], kernel_initializer='uniform',activation="sigmoid"))
model.add(Dense(1, kernel_initializer='uniform',activation="sigmoid")) #this should be mantained
model.compile(optimizer=SGD(lr=0.01),loss='binary_crossentropy')
hist = model.fit(X_train_scaled, y_train, epochs=100, verbose=1, validation_data=(X_val_scaled, y_val))

Finalmente compare a través de una métrica de desempeño sobre el conjunto de pruebas, en este caso como trabajamos un problema desbalanceado, mida *f1 score weighted*, comente sobre esta decisión ¿Es esperable la diferencia entre relu y sigmoidal en base a los gráficos realizados?

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test,model_sigmoid.predict_classes(X_test_scaled),average='weighted')
f1_score(y_test,model_relu.predict_classes(X_test_scaled),average='weighted')

**d)** Repita **c)** variando la tasa de aprendizaje (*learning rate*) en un rango sensible. Comente. Si observa divergencia durante el entrenamiento, determine si esto ocurre para cada repetición del experimento o para alguna de las dos funciones de activación experimentadas.

In [ ]:
import numpy as np
n_lr = 20
lear_rate = np.linspace(0,1,n_lr)

**e)** Entrene los modelos considerados en c) usando *progressive decay*. Compare y comente.

In [ ]:
n_decay = 10
lear_decay = np.logspace(-6,0,n_decay)
sgd = SGD(lr=0.2, decay=1e-6)

**f)** Entrene los modelos considerados en c) usando *momentum* [[3]](#refs). Experimente usando *momentum* clásico y *momentum* de Nesterov. ¿Observa un mejor resultado final? ¿Observa una mayor velocidad de convergencia sobre el conjunto de entrenamiento? ¿Sobre el conjunto de validación?

In [ ]:
split_space = 21
momentum = np.linspace(0,1,split_space)
sgd = SGD(lr=0.01,momentum=0.9,nesterov=False)

**g)** Vuelva a entrenar los modelos considerados en c) utilizando SGD en mini-*batches*. Experimente con diferentes tamaños del *batch*. Comente sobre eficacia de convergencia (converge en menor número de *epochs*) *vs* eficiencia computacional (tiempo en converger o por *epoch*).

In [ ]:
n_batches = 21
batch_sizes = np.round(np.linspace(1,X_train_scaled.shape[0],n_batches))
model.fit(X_train_scaled,y_train,batch_size=50,epochs=100,validation_data=(X_val_scaled, y_val))

**h)** Entrene los modelos obtenidos en c) utilizando estrategias modernas para adaptar la tasa de aprendizaje. Compare los desempeños de *adagrad, adadelta, RMSprop* y *adam*, ofrecidos en __[keras optimizer](https://keras.io/optimizers/)__. ¿Se observa en algún caso un mejor resultado final? ¿Se observa en algún caso una mayor velocidad de convergencia sobre el dataset de entrenamiento? ¿Sobre el dataset de validación?

In [ ]:
from keras.optimizers import SGD, Adam, RMSprop, Adagrad, Adadelta
moptimizer = Adagrad(lr=0.01)
model.compile(optimizer=moptimizer)
model.fit(X_train_scaled,y_train,batch_size=bs,epochs=100,validation_data=(X_val_scaled, y_val))

**i)** Entrene los modelos obtenidos en c) utilizando regularizadores clásicos $l_1$ y $l_2$ (*weight decay*). Compare los desempeños de prueba obtenidos antes y después de regularizar. Experimente con distintos valores del parámetro de regularización y comente. Además evalúe el efecto de regularizar solo la primera capa *vs* la segunda, comente. *Recuerde que la regularización se debe añadir a cada capa separadamente* ¿Dónde le parece más importante aplicar?

In [ ]:
from keras.layers import Activation
from keras.regularizers import l1,l2
model = Sequential()
model.add(Dense(256,input_dim=X_train_scaled.shape[1],kernel_initializer='uniform',kernel_regularizer=l2(0.01)))
model.add(Activation('sigmoid')) #and relu
model.add(Dense(1, kernel_initializer='uniform',kernel_regularizer=l2(0.01)))
model.add(Activation('sigmoid'))

**j)** Elija uno de los dos modelos definidos en c) y experimente con modificar la función objetivo, que hasta ahora se lo hemos dejado a keras definir la típica *binary cross entropy*, en pos de algún objetivo que decida plantearse, por ejemplo aumentar la velocidad de convergencia (reducir más rápida la función objetivo), obtener mejor desempeño en *f1 score* u algún otro.  
*Se dejan algunos ejemplos*

In [ ]:
1. Focal loss [[4]](#refs)
2. Cross entropy + Min entropy
3. Cross entropy + Max entropy
4. Kullback Leibler (KL) ó Jensen Shannon (JS) divergence [[5]](#refs)  
5. Weighted Cross entropy  
    
from keras import backend as K
def custom_loss(value): #ejemplo
    def loss(y_true, y_pred):
        y_true = K.clip(y_true, K.epsilon(), 1) 
        y_pred = K.clip(y_pred, K.epsilon(), 1)
        return K.sum(y_true * K.log(y_true / y_pred), axis=-1) #KL-divergence
        ...#return value*K.mean(K.square(y_pred - y_true), axis=-1) #MSE
    return loss
model.compile(loss=custom_loss(1),optimizer=opt) #example of use

<a id="refs"></a>
## Referencias
[1] Borucki, W. J., Koch, D. G., Basri, G., Batalha, N., Boss, A., Brown, T. M., ... & Dunham, E. W. (2011). *Characteristics of Kepler planetary candidates based on the first data set*. The Astrophysical Journal, 728(2), 117. also in: https://exoplanetarchive.ipac.caltech.edu/index.html  
[2] Bugueno, M., Mena, F., & Araya, M. *Refining Exoplanet Detection Using Supervised Learning and Feature Engineering*.  
[3] Sutskever, I., Martens, J., Dahl, G. E., & Hinton, G. E. (2013). *On the importance of initialization and momentum in deep learning*. ICML (3), 28(1139-1147), 5.  
[4] Lin, T. Y., Goyal, P., Girshick, R., He, K., & Dollár, P. (2017). *Focal loss for dense object detection*. In Proceedings of the IEEE international conference on computer vision (pp. 2980-2988).  
[5] Chen, P., Chen, Y., & Rao, M. (2008). *Metrics defined by Bregman divergences: Part 2*. Communications in Mathematical Sciences, 6(4), 927-948.  
[6] Glorot, X., & Bengio, Y. (2010, March). *Understanding the difficulty of training deep feedforward neural networks*. In Proceedings of the thirteenth international conference on artificial intelligence and statistics (pp. 249-256).   
[7] He, K., Zhang, X., Ren, S., & Sun, J. (2015). *Delving deep into rectifiers: Surpassing human-level performance on imagenet classification*. In Proceedings of the IEEE international conference on computer vision (pp. 1026-1034).    
[8] Srivastava, N., Hinton, G., Krizhevsky, A., Sutskever, I., & Salakhutdinov, R. (2014). *Dropout: a simple way to prevent neural networks from overfitting*. The Journal of Machine Learning Research, 15(1), 1929-1958.  
[9] Ioffe, S., & Szegedy, C. (2015). Batch normalization: *Accelerating deep network training by reducing internal covariate shift*. arXiv preprint arXiv:1502.03167.  
[10] Krizhevsky, A., Nair, V., & Hinton, G. (2014). *The CIFAR-10 dataset*. online: http://www.cs.toronto.edu/kriz/cifar.html , 4.  
[11] Bengio, Y. (2012, June). *Deep learning of representations for unsupervised and transfer learning*. In Proceedings of ICML Workshop on Unsupervised and Transfer Learning (pp. 17-36).  
[12] Simonyan, K., & Zisserman, A. (2014). *Very deep convolutional networks for large-scale image recognition*. arXiv preprint arXiv:1409.1556.  
[13] Ruder, S. (2017). *An overview of multi-task learning in deep neural networks*. arXiv preprint arXiv:1706.05098.  